In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"  # 0 = all logs, 1 = filter INFO, 2 = filter WARNING, 3 = filter ERROR
import warnings
warnings.filterwarnings("ignore")

from traning_classes import DataManager, ModelPerformanceIndicators
from model_architectures import I5_architecture, I20_architecture, I60_architecture
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [2]:
model_names = [
    # "I5-R5", Già fatto
    # "I5-R20",
    # "I5-R60",
    # "I20-R5",
    # "I20-R20",
    # "I20-R60",
    "I60-R5",
    "I60-R20",
    "I60-R60"
]

In [3]:
import gc
import tensorflow as tf

def train_pipeline(model_name):
    try:
        # === GET DATA ===
        dm = DataManager()
        image_dir = f"../_images/{model_name}/traning"
        dm.load_images(image_dir)
        X_train, X_test, y_train, y_test = dm.split_data()

        # === GET MODEL ARCHITECTURE ===
        if model_name.startswith("I5"):
            model = I5_architecture()
        elif model_name.startswith("I20"):
            model = I20_architecture()
        elif model_name.startswith("I60"):
            model = I60_architecture()
        else:
            raise ValueError("Model name not recognized")

        # === TRAINING ===
        early_stopping = EarlyStopping(
            monitor='val_loss',
            mode='min',
            patience=10,
            restore_best_weights=True
        )
        reduce_lr = ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.1,
            patience=2,
            min_lr=1e-6
        )
        callbacks = [early_stopping, reduce_lr]

        history = model.fit(
            x=X_train,
            y=y_train,
            batch_size=128,
            epochs=40,
            validation_split=0.3,
            callbacks=callbacks,
            verbose=2
        ).history

        # === EVALUATE PERFORMANCE ===
        test_predictions = model.predict(X_test, verbose=0)
        ModelPerformanceIndicators.save_pdf(test_predictions, y_test, model_name)

        # === SAVE MODEL ===
        model_filename = f'../models/{model_name}.keras'
        model.save(model_filename)

    finally:
        # === CLEAN UP MEMORY ===
        del X_train, X_test, y_train, y_test
        del history, test_predictions
        del model
        gc.collect()  # Python garbage collection
        tf.keras.backend.clear_session()  # Clears TensorFlow session/graph

In [4]:
for model_name in model_names:
    print(f"Training model: {model_name}")
    train_pipeline(model_name)

Training model: I60-R5
TRUE: 121686 images with shape (96, 180)
FALSE: 99402 images with shape (96, 180)
Balanced counts: {0: 99402, 1: 99402}
Training set: 139162 samples
Test set: 59642 samples
Image shape: (96, 180)


I0000 00:00:1755094587.199774 1232822 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1755094587.199870 1232822 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/40
762/762 - 1752s - 2s/step - accuracy: 0.5066 - loss: 11.6086 - val_accuracy: 0.5059 - val_loss: 1.0731 - learning_rate: 0.0100
Epoch 2/40
762/762 - 1807s - 2s/step - accuracy: 0.5054 - loss: 1.3185 - val_accuracy: 0.5055 - val_loss: 0.8647 - learning_rate: 0.0100
Epoch 3/40


UnboundLocalError: cannot access local variable 'history' where it is not associated with a value